In [1]:
from pathlib import Path 
import os
import torch
from transformers import MarianMTModel, MarianTokenizer

/Users/ball/Documents/workspace/transformer-tutorial/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(f'@@Configuration START@@')

@@Configuration START@@


In [3]:
# Define configuration 
batch_size = 64
d_model = 400 
n_head = 8
max_len = 128
ffn_hidden = 1200
n_layers=4 
drop_prob=0.1
epochs=50
init_lr = 0.00 # having warmup step
eps = 1e-9
weight_decay = 5e-4
warmup_steps=500
clip = 1

print(f'batch_size: {batch_size}')
print(f'd_model: {d_model}')
print(f'n_head: {n_head}') 
print(f'max_len: {max_len}') 
print(f'ffn_hidden: {ffn_hidden}')
print(f'n_layers: {n_layers}')
print(f'drop_prob: {drop_prob}')
print(f'epochs: {epochs}')
print(f'init_lr: {init_lr}')
print(f'weight_decay: {weight_decay}')
print(f'clip: {clip}')

batch_size: 32
d_model: 32
n_head: 8
max_len: 128
ffn_hidden: 32
n_layers: 3
drop_prob: 0.1
epochs: 300
init_lr: 0.01
weight_decay: 0.0005
clip: 1


In [14]:
# Configure Device 


if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA as device")
else:
    # Check that MPS is available
    if not torch.backends.mps.is_available():
        if not torch.backends.mps.is_built():
            print("MPS not available because the current PyTorch install was not "
                  "built with MPS enabled.")
        else:
            print("MPS not available because the current MacOS version is not 12.3+ "
                  "and/or you do not have an MPS-enabled device on this machine.")
        device = torch.device("cpu")
        print("Using CPU as device")
    else:
        device = torch.device("mps")
        print("Using MPS as device")
    
torch.set_default_device(device)

Using MPS as device


In [4]:
# Define tokenizers
TOKENIZERS_PARALLELISM = True

kr_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-ko-en')
en_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-ko-en')

enc_voc_size = kr_tokenizer.vocab_size
dec_voc_size = en_tokenizer.vocab_size

print(f'Using kr_tokenizer: Helsinki-NLP/opus-mt-ko-en')
print(f'kr_tokenizer_voc_size(enc_voc_size): {enc_voc_size}')

print(f'Using en_tokenizer: Helsinki-NLP/opus-mt-ko-en')
print(f'en_tokenizer_voc_size(dec_voc_size): {dec_voc_size}')

# Define some variables that are going to be used in future
src_pad_token = kr_tokenizer.pad_token_id
src_eos_token = kr_tokenizer.eos_token_id

trg_pad_token = en_tokenizer.pad_token_id
trg_sos_token = en_tokenizer.bos_token_id
trg_eos_token = en_tokenizer.eos_token_id

print(f'src_pad_token: {src_pad_token}')
print(f'src_eos_token: {src_eos_token}')
print(f'trg_pad_token: {trg_pad_token}')
print(f'trg_sos_token: {trg_sos_token}')
print(f'trg_eos_token: {trg_eos_token}')

/Users/ball/Documents/workspace/transformer-tutorial/.venv/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Using kr_tokenizer: Helsinki-NLP/opus-mt-ko-en
kr_tokenizer_voc_size(enc_voc_size): 65001
Using en_tokenizer: Helsinki-NLP/opus-mt-ko-en
en_tokenizer_voc_size(dec_voc_size): 65001
src_pad_token: 65000
src_eos_token: 0
trg_pad_token: 65000
trg_sos_token: None
trg_eos_token: 0


In [5]:
# Test tokenizers 
tmp_kr_sentence = "오늘 하교길에 길고양이를 보았는데, 너무 귀여워서 집에 데려가고 싶었다. 하지만 그러지는 않았다."
tmp_en_sentence = "The cat I saw during heading home today was so cute, that I wanted to bring it to home."

tmp_kr_tokenized = kr_tokenizer(tmp_kr_sentence, add_special_tokens=True, padding="max_length", max_length=256, truncation=True)
# tmp_en_tokenized = en_tokenizer(tmp_en_sentence, add_special_tokens=True, padding="max_length", max_length=256, truncation=True)

# print(tmp_kr_tokenized)
# print(kr_tokenizer.convert_ids_to_tokens(tmp_kr_tokenized.input_ids))
# print(en_tokenizer.convert_ids_to_tokens(tmp_en_tokenized.input_ids))
# print(tmp_kr_tokenized.input_ids)
# print(kr_tokenizer.decode(tmp_kr_tokenized.input_ids, skip_special_tokens=True))

# check if both tokenizer has pad token 
# print(kr_tokenizer.pad_token)
# print(en_tokenizer.pad_token)

[1114, 420, 2331, 573, 64, 2985, 21280, 79, 9, 31635, 3, 948, 3961, 580, 21323, 2245, 8522, 85, 12669, 161, 2, 199, 8693, 55, 16036, 2, 0, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 

In [21]:
# Define path configuration for the project 
project_dir = Path(os.getcwd()).parent
rawdata_dir = project_dir / "data"
data_dir = project_dir / "preprocessed"
model_dir = project_dir / "models"

rawdata_dir.mkdir(parents=True, exist_ok=True)
data_dir.mkdir(parents=True, exist_ok=True)
model_dir.mkdir(parents=True, exist_ok=True)

print(f'project_dir: {project_dir}')
print(f'rawdata_dir: {rawdata_dir}')
print(f'data_dir: {data_dir}')

project_dir: /Users/ball/Documents/workspace/transformer-tutorial
rawdata_dir: /Users/ball/Documents/workspace/transformer-tutorial/data
data_dir: /Users/ball/Documents/workspace/transformer-tutorial/preprocessed


In [ ]:
print(f'@@Configuration END@@')